In [ ]:
!pip install catboost
!pip install optuna

In [50]:
# ===MODULES===
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mlxtend.plotting import scatterplotmatrix

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.metrics import make_scorer

from xgboost import XGBRegressor
from catboost import CatBoostRegressor
# 카테고리 데이터가 있을때 성능이 좋음
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.ensemble import GradientBoostingRegressor, VotingRegressor, RandomForestRegressor

In [171]:
# ===DATA===

train = pd.read_csv('/content/drive/MyDrive/gh/kaggle/dacon/bicycle/dataset/train.csv')
test = pd.read_csv('/content/drive/MyDrive/gh/kaggle/dacon/bicycle/dataset/test.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/gh/kaggle/dacon/bicycle/dataset/sample_submission.csv')
train.head()

,date_time,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,number_of_rentals
0,2018-04-01,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,22994
1,2018-04-02,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,28139
2,2018-04-03,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,26817
3,2018-04-04,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,26034
4,2018-04-05,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,2833


In [172]:
# ===UTILS===

# 1. seperate datetime
def seperate_datetime(dataframe):
    year = []
    month = []
    day = []

    for date in dataframe.date_time:
        year_point, month_point, day_point = date.split('-') # - 기준으로 string을 나누고 list로 만듦 ex) '2016-04-01' -> ['2016', '04', '01']
        year.append(int(year_point)-2017)
        month.append(int(month_point)-3)
        day.append(int(day_point))
    return year, month, day


# 2. Calculate NMAE
def NMAE(true, pred):
    return np.mean(np.abs(true-pred) / true)



In [173]:
year, month, day = seperate_datetime(train)

train['year'] = year
train['month'] = month
train['day'] = day

X = train.drop(['date_time', 'number_of_rentals'], axis=1)
y = train.number_of_rentals

In [174]:
X

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,day
0,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,1,1,1
1,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,1,1,2
2,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,1,1,3
3,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,1,1,4
4,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...
268,228.662,3.980,0.223,2.271,78.378,20.500,27.526,36.486,3,3,26
269,207.770,2.865,0.081,1.794,78.412,20.812,28.842,21.081,3,3,27
270,282.568,1.730,0.000,1.820,72.736,21.000,29.053,7.297,3,3,28
271,137.027,2.257,0.088,2.043,70.473,19.625,26.000,15.541,3,3,29


In [179]:
X_combination = X.copy()

In [181]:
week_day = pd.to_datetime(train['date_time']).dt.day_name()
# le = LabelEncoder()
# le.fit(week_day)
# X_combination['week_day'] = le.transform(week_day)
# X_combination = pd.concat([X_combination,pd.get_dummies(week_day)],axis=1)
X_combination['week_day'] = week_day

wd_idx = {'Monday': 0, 'Tuesday' : 1, 'Wednesday' : 2, 'Thursday' : 3, 'Friday' : 4, 'Saturday' : 5, 'Sunday' : 6}

def wd_cat2float(cat):
  return wd_idx[cat]

X_combination['week_day'] = X_combination['week_day'].map(wd_cat2float)
# week_day = pd.to_datetime(train['date_time']).dt.day_name()
# train['week_day'] = week_day
# train.head()
X_combination

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,day,week_day
0,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,1,1,1,6
1,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,1,1,2,0
2,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,1,1,3,1
3,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,1,1,4,2
4,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,1,1,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...
268,228.662,3.980,0.223,2.271,78.378,20.500,27.526,36.486,3,3,26,4
269,207.770,2.865,0.081,1.794,78.412,20.812,28.842,21.081,3,3,27,5
270,282.568,1.730,0.000,1.820,72.736,21.000,29.053,7.297,3,3,28,6
271,137.027,2.257,0.088,2.043,70.473,19.625,26.000,15.541,3,3,29,0


In [182]:
X_combination = X_combination.drop(['wind_direction', 'Precipitation_Probability','day'], axis=1)

X_combination

,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,year,month,week_day
0,4.000,0.000,3.050,75.000,12.600,21.000,1,1,6
1,2.950,0.000,3.278,69.833,12.812,19.000,1,1,0
2,2.911,0.000,2.690,74.879,10.312,15.316,1,1,1
3,3.692,0.425,3.138,71.849,8.312,12.368,1,1,2
4,4.000,0.723,3.186,73.784,5.875,10.421,1,1,3
...,...,...,...,...,...,...,...,...,...
268,3.980,0.223,2.271,78.378,20.500,27.526,3,3,4
269,2.865,0.081,1.794,78.412,20.812,28.842,3,3,5
270,1.730,0.000,1.820,72.736,21.000,29.053,3,3,6
271,2.257,0.088,2.043,70.473,19.625,26.000,3,3,0


In [185]:
vt = CatBoostRegressor()
vt.fit(X_combination, y)

y_hat=vt.predict(X_combination)

score = NMAE(y, y_hat)

print(f'NMAE: {score}')

Learning rate set to 0.03335
0:	learn: 27023.6832368	total: 3.68ms	remaining: 3.67s
1:	learn: 26514.9947434	total: 5.08ms	remaining: 2.54s
2:	learn: 25939.5167899	total: 6.43ms	remaining: 2.14s
3:	learn: 25392.6610533	total: 7.63ms	remaining: 1.9s
4:	learn: 24913.3753470	total: 8.99ms	remaining: 1.79s
5:	learn: 24456.7786963	total: 10.3ms	remaining: 1.71s
6:	learn: 23942.7353743	total: 11.1ms	remaining: 1.57s
7:	learn: 23449.2314228	total: 12.3ms	remaining: 1.52s
8:	learn: 23035.9822581	total: 13.5ms	remaining: 1.49s
9:	learn: 22566.8409952	total: 14.7ms	remaining: 1.46s
10:	learn: 22112.9390678	total: 15.9ms	remaining: 1.43s
11:	learn: 21732.7988889	total: 17.1ms	remaining: 1.41s
12:	learn: 21281.1253107	total: 18.4ms	remaining: 1.4s
13:	learn: 20887.1850672	total: 19.7ms	remaining: 1.39s
14:	learn: 20506.0875545	total: 21.4ms	remaining: 1.4s
15:	learn: 20133.7051566	total: 22.6ms	remaining: 1.39s
16:	learn: 19739.6235585	total: 23.9ms	remaining: 1.38s
17:	learn: 19406.8019628	total: 

In [78]:
# bc_RF = RandomForestRegressor()
# bc_Cat = CatBoostRegressor()
# bc_XGB = XGBRegressor()
# bc_LGBM = LGBMRegressor()

# vt = VotingRegressor([
#                       ('bc_Cat', bc_Cat), 
#                       ('bc_XGB', bc_XGB), 
#                       ('bc_LGBM', bc_LGBM),
#                       ('bc_RF', bc_RF)
#                       ])


In [107]:
vt = XGBRegressor()
X_train, X_valid, y_train, y_valid = train_test_split(X_combination, y, test_size=0.1)

def objective(trial: Trial) -> float:
    params_xgb = {
        'objective':'reg:squarederror',
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 4000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20]),
        'random_state': trial.suggest_categorical('random_state', [24, 48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    
    

    model = XGBRegressor(**params_xgb)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        early_stopping_rounds=100,
        verbose=False,
    )

    xgb_pred = model.predict(X_valid)
    score = NMAE(y_valid, xgb_pred)
    
    return score

In [ ]:
sampler = TPESampler(seed=42)
bc_study = optuna.create_study(
    study_name="xgb_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
bc_study.optimize(objective, n_trials=150)
print("Best NMAE:", bc_study.best_value)
print("Best trial:", bc_study.best_trial.params)

In [109]:
optuna.visualization.plot_optimization_history(bc_study)

In [110]:
optuna.visualization.plot_param_importances(bc_study)

In [111]:
best_params = bc_study.best_trial.params

In [112]:
bc_XGB = XGBRegressor(**best_params)

In [113]:
bc_XGB.fit(X_train, y_train)

[08:57:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=0.09979772050090538, base_score=0.5, booster='gbtree',
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.9,
             gamma=0, importance_type='gain', lambda=0.3040076472763329,
             learning_rate=0.018, max_delta_step=0, max_depth=13,
             min_child_weight=3, missing=None, n_estimators=100, n_jobs=1,
             nthread=None, objective='reg:linear', random_state=48, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1.0, verbosity=1)

In [114]:
y_hat=bc_XGB.predict(X_train)

score = NMAE(y_train, y_hat)

print(f'NMAE: {score}')

NMAE: 0.1815558328806289


In [183]:
# ===TEST===
year, month, day = seperate_datetime(test) #날짜 숫자로 분리 후 

#새로운 컬럼을 생성
test['year'] = year 
test['month'] = month
test['day'] = day

# date_time은 제거합니다.
test_X = test.drop('date_time', axis = 1)


# 요일 정보 추가

week_day = pd.to_datetime(test['date_time']).dt.day_name()
# le = LabelEncoder()
# le.fit(week_day)
test_X['week_day'] = week_day

test_X['week_day'] = test_X['week_day'].map(wd_cat2float)

# week_day = pd.to_datetime(test['date_time']).dt.day_name()
# test_X = pd.concat([test_X,pd.get_dummies(week_day)],axis=1)
test_X

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,day,week_day
0,108.833,3.000,0.000,2.900,28.333,11.800,20.667,18.333,4,1,1,3
1,116.717,3.850,0.000,2.662,46.417,12.000,19.000,28.500,4,1,2,4
2,82.669,4.000,0.565,2.165,77.258,8.875,16.368,52.847,4,1,3,5
3,44.123,3.466,0.466,3.747,63.288,6.250,17.368,37.671,4,1,4,6
4,147.791,1.500,0.000,1.560,48.176,7.188,18.684,4.459,4,1,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
86,177.149,3.980,0.223,1.066,74.628,20.312,28.579,36.486,4,3,26,5
87,138.723,2.777,0.135,1.290,70.236,20.812,29.000,18.378,4,3,27,6
88,111.095,3.338,1.270,1.692,70.338,21.000,28.789,35.946,4,3,28,0
89,171.622,3.270,0.595,1.470,70.473,21.000,29.421,27.770,4,3,29,1


In [184]:
test_X = test_X.drop(['wind_direction', 'Precipitation_Probability','day'], axis=1)
test_X

,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,year,month,week_day
0,3.000,0.000,2.900,28.333,11.800,20.667,4,1,3
1,3.850,0.000,2.662,46.417,12.000,19.000,4,1,4
2,4.000,0.565,2.165,77.258,8.875,16.368,4,1,5
3,3.466,0.466,3.747,63.288,6.250,17.368,4,1,6
4,1.500,0.000,1.560,48.176,7.188,18.684,4,1,0
...,...,...,...,...,...,...,...,...,...
86,3.980,0.223,1.066,74.628,20.312,28.579,4,3,5
87,2.777,0.135,1.290,70.236,20.812,29.000,4,3,6
88,3.338,1.270,1.692,70.338,21.000,28.789,4,3,0
89,3.270,0.595,1.470,70.473,21.000,29.421,4,3,1


In [186]:
# train data로 학습시킨 모델에 test_X데이터를 넣고 예측합니다.
test_yhat = vt.predict(test_X)

# submission dataFrame 완성
sample_submission['number_of_rentals'] = test_yhat

# 제출 파일 생성
sample_submission.to_csv('/content/drive/MyDrive/gh/kaggle/dacon/bicycle/dataset/CATjebal.csv', index= False)

In [ ]:
def objective(trial: Trial) -> float:
    params_xgb = {
        "random_state": 42,
        "verbosity": -1,
        "learning_rate": 0.1,
        "n_estimators": trial.suggest_int("n_estimators", 100, 5000, step = 100),
        "objective": "regression",
        "metric": "mean_squared_error",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 0.5),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 9e-2),
        "max_depth": trial.suggest_int("max_depth", 1, 50),
        "num_leaves": trial.suggest_int("num_leaves", 2, 128),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "bagging_fraction": min(trial.suggest_float("bagging_fraction", 0.4, 1.0 + 1e-12), 1),
        "bagging_freq": trial.suggest_int("bagging_freq", 0, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 80),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
    }
    
    X_train, X_valid, y_train, y_valid = train_test_split(x_lu_train, y_lu_train, test_size=0.2)

    model = LGBMRegressor(**params_lgb)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        early_stopping_rounds=100,
        verbose=False,
    )

    lgb_pred = model.predict(X_valid)
    score = mean_squared_error(y_valid, lgb_pred)
    
    return score

In [ ]:
X_combination = X.copy()

In [ ]:
# ===FEATURE ENGINEERING===

# 요일 정보 추가
week_day = pd.to_datetime(train['date_time']).dt.day_name()
le = LabelEncoder()
le.fit(week_day)
X_combination['week_day'] = le.transform(week_day)

# 1. "일교차가 너무 큰 날씨"를 알기 위한 정보
X_combination['temp_diff_info'] = X_combination['high_temp'] - X_combination['low_temp']

# 2. "덥고 습한 날씨"를 알기 위한 정보
X_combination['sweat_info'] = X_combination['high_temp'] * X_combination['humidity'] 

# 3. "춥고 바람부는 날씨"를 알기 위한 정보
X_combination['cold_info'] = X_combination['low_temp'] * X_combination['wind_speed']


In [ ]:
X_combination = X_combination.drop(['wind_direction', 'Precipitation_Probability','humidity'], axis=1)
col_list = X_combination.columns
# 이중 for문을 사용하여 변수 자기 자신의 제곱과 두 변수간의 곱이라는 새로운 변수를 추가합니다.
for i in range(len(col_list)):
    for j in range(i, len(col_list)):
        X_combination[f'{col_list[i]}*{col_list[j]}'] = X_combination[col_list[i]] * X_combination[col_list[j]]

X_2=X_combination

In [ ]:
# # train-val split
# X_train, X_val, y_train, y_val = train_test_split(X_2, 
#                                                   y, 
#                                                   test_size=0.25, 
#                                                   random_state=42,
#                                                   shuffle=False)

In [ ]:
# 모델 선언
bc_RF = RandomForestRegressor()
bc_Cat = CatBoostRegressor()
bc_XGB = XGBRegressor()
bc_LGBM = LGBMRegressor()
# bc_GBM = GradientBoostingRegressor(n_estimators=4000,alpha=0.01)
# bc_Ridge = Ridge()
# bc_Lasso = Lasso()

# 보팅 생성
vt = VotingRegressor([
                      ('bc_Cat', bc_Cat), 
                      ('bc_XGB', bc_XGB), 
                      ('bc_LGBM', bc_LGBM),
                      ('bc_RF', bc_RF),
                      ('bc_GBM', bc_GBM),
                      ('bc_Ridge', bc_Ridge),
                      ('bc_Lasso', bc_Lasso)
                      ])

# 훈련 + cross_val_score 검증

scores = cross_validate(estimator=vt,
                        X=X_2,
                        y=y,
                        scoring=make_scorer(NMAE),
                        cv=10,
                        n_jobs=-1)

print(scores['test_score'])

[0.37530979 0.65374004 0.10363103 0.19389011 0.08709973 0.19294367
 0.12696399 0.11218676 0.36763133 0.13173211]


In [ ]:
vt.fit(X_2, y)

In [ ]:
# 예측
y_hat = vt.predict(X_2)

score = NMAE(y, y_hat)

print(f'NMAE: {score}')

NMAE: 0.05921272072044324


In [ ]:
# ===TEST===

year, month, day = seperate_datetime(test) #날짜 숫자로 분리 후 

#새로운 컬럼을 생성
test['year'] = year 
test['month'] = month
test['day'] = day

# date_time은 제거합니다.
test_X = test.drop('date_time', axis = 1)


# 요일 정보 추가

week_day = pd.to_datetime(test['date_time']).dt.day_name()
le = LabelEncoder()
le.fit(week_day)
test_X['week_day'] = le.transform(week_day)

# 1. "일교차가 너무 큰 날씨"를 알기 위한 정보
test_X['temp_diff_info'] = test_X['high_temp'] - test_X['low_temp']

# 2. "덥고 습한 날씨"를 알기 위한 정보
test_X['sweat_info'] = test_X['high_temp'] * test_X['humidity'] 

# 3. "춥고 바람부는 날씨"를 알기 위한 정보
test_X['cold_info'] = test_X['low_temp'] * test_X['wind_speed'] 

In [ ]:
test_X = test_X.drop(['wind_direction', 'Precipitation_Probability','humidity'], axis=1)


In [ ]:
col_list = test_X.columns

for i in range(len(col_list)):
    for j in range(i, len(col_list)):
        test_X[f'{col_list[i]}*{col_list[j]}'] = test_X[col_list[i]] * test_X[col_list[j]]

In [ ]:
# train data로 학습시킨 모델에 test_X데이터를 넣고 예측합니다.
test_yhat = vt.predict(test_X)

# submission dataFrame 완성
sample_submission['number_of_rentals'] = test_yhat

# 제출 파일 생성
sample_submission.to_csv('/content/drive/MyDrive/gh/kaggle/dacon/bicycle/dataset/ridgelasso2.csv', index= False)

In [ ]:
# 1.다중회귀 앙상블
# 2.randomserch cv 적용
# 3.아래 계산해서 칼럼 추가
# weekend 토요일 일요일이 아니고 날씨도 좋았지만 대여량이 낮은날.
# discomfort index 불쾌지수
# sensible temperature 체감온도
# 4.wb,optuna 적용 가능 탐색
# 유튜브 데이터 홀릭



# 1. 정규화 표준화
# 2. 요일컬럼 추가
# 3. 불쾌지수
# 4. 체감온도
# 5. 논문탐색
# 6. 앙상블 구성
# 7. 옵튜나 적용
# 8. 일교차
# 9. 컬럼 중요도


# # 1. "일교차가 너무 큰 날씨"를 알기 위한 정보
# X_combination['temp_diff_info'] = X_combination['high_temp'] - X_combination['low_temp']

# # 2. "덥고 습한 날씨"를 알기 위한 정보
# X_combination['sweat_info'] = X_combination['high_temp'] * X_combination['humidity'] 

# # 3. "춥고 바람부는 날씨"를 알기 위한 정보
# X_combination['cold_info'] = X_combination['low_temp'] * X_combination['wind_speed'] 